In [126]:
%matplotlib inline
import pandas as pd
import datetime as dt
import random
import itertools
import pdb
import time
from dateutil.parser import parse
from experiment.models import Models, Parameters
from experiment.models import get_metadata_file, get_pubdate_index

mt = get_metadata_file()
p = Parameters()
p.q = "Mitch Landrieu"

start_time = time.time()
results = Models.get_results(p)
print "query took {}".format(start_time - time.time())

start_time = time.time()
facets, aliases = Models.get_facets(p, results, 3)
print "facets took {}".format(start_time - time.time())

metadata = [mt[r] for r in results]
q_pubdates = [parse(h["pubdate"]) for h in metadata]

pub_index = get_pubdate_index()

def make_dataframe(p, facets, results):
    start_time = time.time()
    df = pd.DataFrame()
    df['id'] = results
    df['pd'] = q_pubdates
    df[p.q] = [1 for i in q_pubdates]
    for facet in facets:
        alias = [pub_index[a] for a in aliases[facet]]
        facet_dates = set([i for i in set([i for i in itertools.chain(*alias)]).union(pub_index[facet])])
        gus = [int(p) for p in [p in facet_dates for p in q_pubdates]]
        df[facet] = gus
    print "dataframe took {}".format(start_time - time.time())
    return df

def get_breakdown(df, facet):
    start_time = time.time()
    tmp = df.groupby([df['pd'].map(lambda x: x.year), df[facet]]).sum()
    tmp = tmp[tmp[facet] != 0][facet].unstack(0).fillna(0)
    print "breakdown took {}".format(start_time - time.time())
    return tmp.to_json()

for f in facets:
    df = make_dataframe(p, facets, results)
    # print type(f)
    print f, get_breakdown(df, f)


DEBUG:experiment:got facets. time to filter


query took -1.47689509392
facets took -1.3633658886
dataframe took -0.0109820365906
Department of Justice breakdown took -0.00959897041321
{"2010":{"1":12.0},"2011":{"1":6.0},"2012":{"1":7.0},"2013":{"1":20.0},"2014":{"1":4.0}}
dataframe took -0.00361490249634
Marlin Gusman breakdown took -0.00396299362183
{"2010":{"1":31},"2011":{"1":23},"2012":{"1":8},"2013":{"1":20},"2014":{"1":15},"2015":{"1":1}}
dataframe took -0.0026159286499
Orleans Parish Sheriffs Office breakdown took -0.009437084198
{"2010":{"1":5},"2011":{"1":14},"2012":{"1":4},"2013":{"1":17},"2014":{"1":8},"2015":{"1":2}}
